# Enrutamiento con LLM Router Chain
## Importar modelo de chat

In [19]:
from langchain.prompts import PromptTemplate,SystemMessagePromptTemplate,ChatPromptTemplate,HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import SequentialChain,LLMChain
f=open('../openai_api_key.txt')
api_key=f.read()
llm=ChatOpenAI(openai_api_key=api_key)

# Plantilla de enrutamiento

In [20]:
plantilla_soporte_basico="""
Eres un agente de monitoreo básico para servidores Asterisk. Tu función es interpretar la siguiente consulta del usuario: {input}. Verifica si Asterisk está activo, responde a SIP y no hay errores simples en logs.
"""

In [21]:
plantilla_soporte_avanzado="""
Eres un agente experto en servidores Asterisk. A partir de {input}, analiza estado del servicio, canales activos, tráfico SIP, errores críticos, uso de CPU y calidad de llamadas. Identifica causas probables y sugiere soluciones técnicas si corresponde.
"""

#  Prompts a enrutar

In [22]:
prompts_infos=[
  {
    "name": "asterisk_monitor_basic",
    "description": "Responde preguntas básicas sobre soporte en Asterisk a clientes.",
    "prompt_template":plantilla_soporte_basico
  },
  {
    "name": "asterisk_monitor_advanced",
    "description": "Responde preguntas avanzadas sobre soporte en Asterisk a clientes.",
    "prompt_template":plantilla_soporte_avanzado
  }
]

## Conversation Chain

In [23]:
destination_chains={}
for p_info in prompts_infos:
    name=p_info['name']
    prompt_template=p_info['prompt_template']
    prompt=ChatPromptTemplate.from_template(template=prompt_template)
    chain=LLMChain(
        llm=llm,
        prompt=prompt
    )
    
    destination_chains[name]=chain

In [24]:
destination_chains

{'asterisk_monitor_basic': LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\nEres un agente de monitoreo básico para servidores Asterisk. Tu función es interpretar la siguiente consulta del usuario: {input}. Verifica si Asterisk está activo, responde a SIP y no hay errores simples en logs.\n'), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x110c6fc50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x110c6fa10>, root_client=<openai.OpenAI object at 0x110c0a210>, root_async_client=<openai.AsyncOpenAI object at 0x110a05dd0>, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}),
 'asterisk_monitor_advanced': LLMChain(verbose=False

In [25]:
default_prompt=ChatPromptTemplate.from_template("{input}")
default_chain=LLMChain(
    llm=llm,
    prompt=default_prompt
)

## Multi Routing

In [26]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [27]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

## Destinos de routing

In [28]:
destinations=[f"{p['name']}:{p['description']}" for p in prompts_infos]
destinations_str="\n".join(destinations)

In [29]:
destinations_str

'asterisk_monitor_basic:Responde preguntas básicas sobre soporte en Asterisk a clientes.\nasterisk_monitor_advanced:Responde preguntas avanzadas sobre soporte en Asterisk a clientes.'

## Router prompt

In [30]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [31]:
router_template=MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt=PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser()
)

In [32]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
asterisk_monitor_basic:Responde preguntas básicas sobre soporte en Asterisk a clientes.
asterisk_moni

## Routing chain call


In [33]:
from langchain.chains.router import MultiPromptChain
router_chain=LLMRouterChain.from_llm(llm,router_prompt)

In [34]:
chain=MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

In [37]:
chain.invoke("Cómo puedo activar pjsip?")



> Entering new MultiPromptChain chain...
asterisk_monitor_basic: {'input': 'Cómo puedo activar pjsip?'}
> Finished chain.


{'input': 'Cómo puedo activar pjsip?',
 'text': 'Para activar el soporte de pjsip en Asterisk, primero debes asegurarte de que Asterisk esté activo y funcionando correctamente. Puedes revisar el estado de Asterisk utilizando el siguiente comando en la terminal:\n\n```\nasterisk -rx "core show status"\n```\n\nSi Asterisk está activo, deberías ver un mensaje indicando que el servicio está funcionando. Una vez que has confirmado que Asterisk está activo, puedes verificar si responde a las solicitudes SIP utilizando el siguiente comando:\n\n```\nasterisk -rx "sip show peers"\n```\n\nEsto te mostrará una lista de los peers SIP registrados en Asterisk y te permitirá confirmar que Asterisk está respondiendo a las solicitudes SIP.\n\nPor último, debes revisar los logs de Asterisk en busca de posibles errores que puedan impedir la activación de pjsip. Puedes revisar los logs de Asterisk ubicados en /var/log/asterisk para buscar posibles errores simples que puedan afectar el funcionamiento de pj

In [18]:
chain.invoke("Qué es Asterisk?")



> Entering new MultiPromptChain chain...
asterisk_monitor_basic: {'input': 'Qué es Asterisk?'}
> Finished chain.


{'input': 'Qué es Asterisk?',
 'text': 'Para determinar si Asterisk está activo, puedes ejecutar el siguiente comando en el servidor:\n\n```\nsudo service asterisk status\n```\n\nSi Asterisk está activo, verás un mensaje indicando que está en funcionamiento. Para verificar si Asterisk responde a SIP, puedes utilizar herramientas como `sip show peers` en la consola de Asterisk o utilizar un cliente SIP para realizar una llamada y confirmar si la comunicación se establece correctamente.\n\nPara verificar si no hay errores simples en los logs de Asterisk, puedes revisar el archivo de logs de Asterisk en busca de mensajes de error o advertencia. Puedes acceder a los logs de Asterisk con el siguiente comando:\n\n```\ntail -f /var/log/asterisk/full\n```\n\nBusca por errores como "ERROR" o "WARNING" para identificar posibles problemas en el servidor Asterisk.\n\nSi todo está funcionando correctamente y no hay errores en los logs, puedes informar al usuario que Asterisk está activo, responde a